In [10]:
import sys
import json
params = {}
default = {'training_folder': '../../scenarios/dallas/', 'params': params} # Default execution parameters
manual, argv = (True, default) if 'ipykernel' in sys.argv[0] else (False, dict(default, **json.loads(sys.argv[1])))
print(argv)


{'training_folder': '../../scenarios/atlanta/', 'params': {}}


In [11]:
import os
import time
import geopandas as gpd
import pandas as pd
sys.path.insert(0, r'../../../quetzal') # Add path to quetzal
from sklearn.neighbors import NearestNeighbors
from numba import jit, njit
import numba as nb
from quetzal.model import stepmodel
from shapely.geometry import LineString
from quetzal.io.gtfs_reader.importer import get_epsg
from quetzal.io import excel
on_lambda = bool(os.environ.get('AWS_EXECUTION_ENV'))
num_cores = nb.config.NUMBA_NUM_THREADS
print('num cores:',num_cores)

io_engine= 'pyogrio' if on_lambda else 'pyogrio' #or fiona

num cores: 8


In [12]:
base_folder = argv['training_folder']
input_folder = os.path.join(base_folder,'inputs/')
pt_folder  = os.path.join(input_folder,'pt/')
road_folder = os.path.join(input_folder,'road/')
params_folder = input_folder if on_lambda else '../../inputs/'

output_folder = os.path.join(base_folder,'outputs/')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    
model_folder = os.path.join(base_folder, 'model/')


In [13]:
sm = stepmodel.read_zippedpickles(model_folder +'aon_los')


zones: 100%|██████████| 4/4 [00:00<00:00, 28.44it/s]                 


In [14]:
var = excel.read_var(file=os.path.join(params_folder,'parameters.xlsx'), scenario='base', return_ancestry=False) 

motor_rate = var['generation']['motor_rate'] 
pt_rate = (1-motor_rate)


In [15]:
pop_mobility = var['generation']['pop_mobility']
job_mobility = var['generation']['job_mobility']

sm.zones['emission'] =  sm.zones['population'] * pop_mobility 
sm.zones['attraction'] =  sm.zones['jobs'] 

car = sm.copy()
car.zones['attraction']= car.zones['attraction'] * pop_mobility
car.zones['emission'] *= motor_rate  

pt_captive = sm.copy()
pt_captive.zones['emission']= pt_captive.zones['emission']
pt_captive.zones['emission'] *= pt_rate

In [16]:
imp_matrix = car.car_los[ 
    ['origin', 'destination', 'time']
].set_index(['origin','destination']).unstack()
imp_matrix = imp_matrix.replace(0, var['step_distribution']['time_intrazonal'])

car.step_distribution(deterrence_matrix=imp_matrix) 

In [17]:
imp_matrix = pt_captive.pt_los[ 
    ['origin', 'destination', 'gtime']
].set_index(['origin','destination']).unstack()
imp_matrix = imp_matrix.replace(0, var['step_distribution']['time_intrazonal'])

In [18]:
pt_captive.step_distribution(deterrence_matrix=imp_matrix)

In [19]:
sm.volumes = pd.merge(
    car.volumes,
    pt_captive.volumes,
    on=['origin', 'destination'],
    suffixes=[ '_car_owner', '_pt_captive']
).rename(columns={'volume_car_owner': 'car_owner', 'volume_pt_captive': 'pt_captive'})


In [20]:
sm.sample_volumes(bidimentional_sampling=True, fit_sums=True, sample_weight={'car_owner':50, 'pt_captive':50}, sample_size=None, inplace=True)

In [21]:
if manual:
    
    od_folder =  os.path.join(input_folder,'od/')

    od = od_folder + 'od.geojson'

    od_test = gpd.read_file(od)
    if 'name' not in od_test.columns:
        od_test['name'] = od_test['index']
    od_test['name'] = od_test['name'].fillna(od_test['index'].astype(str))
    from shapely.geometry import Point
    from syspy.spatial.spatial import nearest, agglomerative_clustering, voronoi_diagram_dataframes, add_geometry_coordinates
    od_test['geometry_o'] = od_test['geometry'].apply(lambda g: Point(g.coords[:][0]))
    od_test['geometry_d'] = od_test['geometry'].apply(lambda g: Point(g.coords[:][1]))
    # find nearest node with KNN. nodes are now the origin and destination.
    od_test['geometry'] = od_test['geometry_o']
    neigh = nearest(od_test, sm.zones.to_crs(4326), n_neighbors=1).rename(columns={'ix_one': 'zone_index', 'ix_many': 'node_index'})
    zone_node_dict = neigh.set_index('zone_index')['node_index'].to_dict()
    od_test['origin'] = od_test.index.map(zone_node_dict.get)

    od_test['geometry'] = od_test['geometry_d']
    neigh = nearest(od_test, sm.zones.to_crs(4326), n_neighbors=1).rename(columns={'ix_one': 'zone_index', 'ix_many': 'node_index'})
    zone_node_dict = neigh.set_index('zone_index')['node_index'].to_dict()
    od_test['destination'] = od_test.index.map(zone_node_dict.get)

    od_test = od_test.drop(columns=['geometry_o','geometry_d','geometry'])
    od_test["pt_captive"] = 0.001
    #Add od_test to volumes after sparsification
    sm.volumes = pd.concat([sm.volumes,od_test[["origin","destination","pt_captive"]]]).sort_values(by="pt_captive",ascending = False).drop_duplicates(["origin","destination"])


    sm.volumes= pd.concat([sm.volumes,od_test[["origin","destination","pt_captive"]]]).sort_values(by="pt_captive",ascending = False).drop_duplicates(["origin","destination"])

In [22]:
sm.to_zippedpickles(model_folder +'distribution', only_attributes= ['volumes', 'zone_before_clustering'])


volumes: 100%|██████████| 11/11 [00:00<00:00, 26.49it/s]              


In [23]:
end_of_notebook

NameError: name 'end_of_notebook' is not defined